x1, x2, x3, y
1,  3,  5,  1
2,  4,  4,  0
.           1 
.           1
            0
            
(satır, sütun) => (5, 1) => (5,)
               => (1, 5)

w = weights
b = bias

z = w1.x1 + w2.x2 + w3.x3 + b

sigmoid = 1 / 1 + exp(-z)

y_hat =   1 / 1 + exp(- (w1.x1 + w2.x2 + w3.x3 + b))

0, 1

y_hat >= 0.5: y_hat = 1
y_hat < 0.5: y_hat = 0
    
X (nx, m)
W
b

In [22]:
class binClass():
    
    def __init__(self, num_iter=10000, learning_rate=5e-5):
        self.num_iters = num_iter
        self.learning_rate = learning_rate
        
    def fit(self, X_train, y_train):
        X_train = X_train.T.astype(np.float128)
        y_train = y_train.values.reshape((1,-1)).astype(np.float128)
        
        W, b = self.initParams(X_train.shape[0])
        costs = []
        
        for i in range(self.num_iters):
            grads, cost = self.prop(X_train, y_train, W, b)
            
            dW = grads['dW']
            db = grads['db']
            
            W, b = self.update(W, b, dW, db, self.learning_rate)
            
            if i%1000 == 0:
                costs.append(cost)
                print(f'{i}th iteration, cost: {cost}')

        params = {'W' : W, 
                  'b' : b}
        
        grads = {'dW': dW,
                 'db': db}
        
        self.params = params
        
        return params, grads, cost
    
    def predict(self, X_test):
        X_test = X_test.T.astype(np.float128)
        
        W = self.params['W']
        b = self.params['b']
        
        A = self.forwProp(X_test, W, b)
        preds = np.zeros((1, X_test.shape[1]))
        
        for i in range(A.shape[1]):
            if A[0, i] < 0.5:
                preds[0, i] = 0
            if A[0 ,i] >= .5:
                preds[0, i] = 1
        
        preds = np.squeeze(preds)
        
        return preds
        
    def initParams(self, dim):
        W = np.zeros((dim, 1))
        b = 0
        
        return W, b
    
    def sigmoid(self, Z):
        Z = 1 / (1 + np.exp(-Z))
        
        return (Z)
    
    def forwProp(self, X_train, W, b):
        A = self.sigmoid(np.dot(W.T, X_train) + b)
        
        return A
    
    def calcCost(self, m, Y, A):
        loss = -(np.sum(np.multiply(Y, np.log(A)) + np.multiply((1-Y), np.log(1-A)))) / m
        
        return loss
    
    def calcGrads(self, m, X_train, y_train, A):
        dW = np.dot(X_train, (A-y_train).T) / m
        db = np.sum(A-y_train) / m
        
        return dW, db
    
    def update(self, W, b, dW, db, learning_rate):
        W = W - (learning_rate * dW)
        b = b - (learning_rate * db)
        
        return W, b
    
    def prop(self, X_train, y_train, W, b):
        
        A = self.forwProp(X_train, W, b)
        
        cost = self.calcCost(X_train.shape[0], y_train, A)
        dW, db = self.calcGrads(X_train.shape[0], X_train, y_train, A)
        
        grad = {'dW' : dW,
                'db' : db}
        
        return grad, cost

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

In [24]:
df = pd.read_csv('../data/BinaryClassification/nba_logreg.csv')

In [25]:
df.head()

,Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,...,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,...,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0
2,JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,...,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0
3,Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,...,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0
4,Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,...,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0


In [26]:
df.dropna(axis=0, how='any', inplace=True)

In [27]:
X = df.drop(columns=['Name', 'TARGET_5Yrs'])

In [28]:
y = df['TARGET_5Yrs']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [30]:
scaler = StandardScaler()

In [31]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [32]:
clf = binClass(num_iter=50000, learning_rate=8e-7)

In [33]:
clf.fit(X_train, y_train)

0th iteration, cost: 38.779760680801154
1000th iteration, cost: 38.09237786441654
2000th iteration, cost: 37.52520281902547
3000th iteration, cost: 37.05448798910944
4000th iteration, cost: 36.661052244437535
5000th iteration, cost: 36.32961452101732
6000th iteration, cost: 36.04808531539453
7000th iteration, cost: 35.80692147635303
8000th iteration, cost: 35.59858295910153
9000th iteration, cost: 35.41709482001784
10000th iteration, cost: 35.25770250832443
11000th iteration, cost: 35.11660405673352
12000th iteration, cost: 34.99074324169423
13000th iteration, cost: 34.877650072913724
14000th iteration, cost: 34.7753176231096
15000th iteration, cost: 34.68210662761318
16000th iteration, cost: 34.59667128580959
17000th iteration, cost: 34.517901275499334
18000th iteration, cost: 34.44487620441395
19000th iteration, cost: 34.376829642094435
20000th iteration, cost: 34.313120566636385
21000th iteration, cost: 34.25321057949398
22000th iteration, cost: 34.19664563080986
23000th iteration, 

({'W': array([[ 0.19155875],
         [ 0.06595626],
         [ 0.061133  ],
         [ 0.06163178],
         [ 0.04689715],
         [ 0.10285095],
         [-0.00436145],
         [-0.02263216],
         [ 0.01288685],
         [ 0.0630026 ],
         [ 0.06105033],
         [ 0.0327532 ],
         [ 0.09034046],
         [ 0.06695696],
         [ 0.07880795],
         [ 0.04125034],
         [ 0.04326419],
         [ 0.0849278 ],
         [ 0.04512611]], dtype=float128),
  'b': 0.20663071425139053706},
 {'dW': array([[-2.91751568],
         [ 0.06880879],
         [ 0.07890603],
         [ 0.09638381],
         [ 0.3061735 ],
         [-1.29163157],
         [ 0.13761844],
         [ 0.56388696],
         [-0.48379983],
         [-0.05506746],
         [ 0.02598092],
         [-0.49545442],
         [-0.65863811],
         [-0.0065204 ],
         [-0.26290251],
         [-0.23380107],
         [ 0.11481016],
         [-0.89621104],
         [ 0.29426087]], dtype=float128),
  'db': -

In [34]:
preds = clf.predict(X_test)

In [39]:
accuracy_score(y_test, preds)

0.7255639097744361

In [40]:
confusion_matrix(y_test, preds)

array([[ 71,  21],
       [ 52, 122]])

In [41]:
from sklearn.linear_model import LogisticRegression

In [43]:
clf_sk = LogisticRegression(max_iter=50000)

In [44]:
clf_sk.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
preds_sk = clf_sk.predict(X_test)

In [46]:
accuracy_score(y_test, preds_sk)

0.7518796992481203

In [47]:
confusion_matrix(y_test, preds_sk)

array([[ 59,  33],
       [ 33, 141]])